In [ ]:
import pandas as pd
from pathlib import Path

# Always show all columns when inspecting
pd.set_option("display.max_columns", None)

# Raw data folder path
PROJECT_ROOT = Path("C:/Users/User/Documents/Projects/dbt_rossmann_anayltics").resolve()
DATA_PATH = PROJECT_ROOT / "data_raw"

files = {
    "store": "store.csv",
    "train": "train.csv"
}

dfs = {}

for name, fname in files.items():
    path = DATA_PATH / fname      
    df = pd.read_csv(path)
    dfs[name] = df

In [2]:
# We name our train.csv as sales and other file.csv as stores
sales = dfs["train"].copy()
store = dfs["store"].copy()


In [3]:
# ==================
# SALES (train.csv)
# ==================

# 1) Parse Date
sales["Date"] = pd.to_datetime(sales["Date"], format="%Y-%m-%d", errors="coerce")
bad_dates = int(sales["Date"].isna().sum())
print(f"[CHECK] Unparseable Date rows: {bad_dates:,}")
print("[INFO] Date dtype:", sales["Date"].dtype)

[CHECK] Unparseable Date rows: 0
[INFO] Date dtype: datetime64[ns]


In [4]:
# 2) Normalize StateHoliday (Rossmann gotcha: '0' vs 0)
sales["StateHoliday"] = sales["StateHoliday"].replace(
    {"0": "none", 0: "none", "a": "public", "b": "easter", "c": "christmas"}
)


In [5]:
# 3) Validate DayOfWeek vs derived day-of-week from Date (Mon=1..Sun=7)
# dbt will derive DayOfWeek,so validate and drop
dow_from_date = sales["Date"].dt.dayofweek + 1

mismatch = (
    sales["Date"].notna()
    & sales["DayOfWeek"].notna()
    & (pd.to_numeric(sales["DayOfWeek"], errors="coerce").astype("Int64") != dow_from_date.astype("Int64"))
)

mismatch_count = int(mismatch.sum())
print("[CHECK] DayOfWeek mismatches:", mismatch_count)
# drop DayOfWeek from dataset
if "DayOfWeek" in sales.columns:
    sales = sales.drop(columns=["DayOfWeek"])


[CHECK] DayOfWeek mismatches: 0


In [6]:
# Data validation checks
# Open vs Sales sanity
closed_and_sales = (sales["Open"] == 0) & (sales["Sales"] > 0)
print(f"[CHECK] Open=0 & Sales>0 rows: {int(closed_and_sales.sum()):,}")

# Composite key sanity: (Store, Date) should be unique in train
dup_store_date = int(sales.duplicated(["Store", "Date"]).sum())
print(f"[CHECK] Duplicate (Store, Date) rows in train: {dup_store_date:,}")


[CHECK] Open=0 & Sales>0 rows: 0
[CHECK] Duplicate (Store, Date) rows in train: 0


In [7]:
print(sales.head())

   Store       Date  Sales  Customers  Open  Promo StateHoliday  SchoolHoliday
0      1 2015-07-31   5263        555     1      1         none              1
1      2 2015-07-31   6064        625     1      1         none              1
2      3 2015-07-31   8314        821     1      1         none              1
3      4 2015-07-31  13995       1498     1      1         none              1
4      5 2015-07-31   4822        559     1      1         none              1


In [8]:
print(sales.dtypes)

Store                     int64
Date             datetime64[ns]
Sales                     int64
Customers                 int64
Open                      int64
Promo                     int64
StateHoliday             object
SchoolHoliday             int64
dtype: object


In [9]:

# =========================
# STORE (store.csv) minimal
# =========================

# 1) Basic ID + categoricals
store["Store"] = pd.to_numeric(store["Store"], errors="raise").astype("int64")

# Keep these as categories/strings (dbt handles labels & logic)
if "StoreType" in store.columns:
    store["StoreType"] = store["StoreType"].astype("category")
if "Assortment" in store.columns:
    store["Assortment"] = store["Assortment"].astype("category")
if "PromoInterval" in store.columns:
    store["PromoInterval"] = store["PromoInterval"].astype("string")



In [10]:
# 2) Safe numeric casting (nullable ints where appropriate)
nullable_int_cols = [
    "CompetitionOpenSinceMonth",
    "CompetitionOpenSinceYear",
    "Promo2SinceWeek",
    "Promo2SinceYear",
]
for col in nullable_int_cols:
    if col in store.columns:
        store[col] = pd.to_numeric(store[col], errors="coerce").astype("Int64")

# CompetitionDistance stays numeric float with NaNs
if "CompetitionDistance" in store.columns:
    store["CompetitionDistance"] = pd.to_numeric(store["CompetitionDistance"], errors="coerce")


In [11]:
# 3) Checks only (no filling / no business rules)
# Store must be unique
store_unique = store["Store"].is_unique
print(f"[CHECK] store.Store is unique: {store_unique}")
if not store_unique:
    dup_store = int(store.duplicated(["Store"]).sum())
    print(f"[WARN] Duplicate Store rows: {dup_store:,}")

# Accepted values (log so you can add dbt accepted_values tests)
if "StoreType" in store.columns:
    print("[INFO] StoreType values:", list(pd.Series(store["StoreType"].astype(str).unique()).dropna()))
if "Assortment" in store.columns:
    print("[INFO] Assortment values:", list(pd.Series(store["Assortment"].astype(str).unique()).dropna()))

# Promo2 consistency check: PromoInterval should usually be null when Promo2=0 
if "Promo2" in store.columns and "PromoInterval" in store.columns:
    promo_interval_when_no_promo2 = store.loc[(store["Promo2"] == 0) & (store["PromoInterval"].notna())]
    print(f"[CHECK] PromoInterval present when Promo2=0: {promo_interval_when_no_promo2.shape[0]:,}")

[CHECK] store.Store is unique: True
[INFO] StoreType values: ['c', 'a', 'd', 'b']
[INFO] Assortment values: ['a', 'c', 'b']
[CHECK] PromoInterval present when Promo2=0: 0


In [12]:
print(store.head())

   Store StoreType Assortment  CompetitionDistance  CompetitionOpenSinceMonth  \
0      1         c          a               1270.0                          9   
1      2         a          a                570.0                         11   
2      3         a          a              14130.0                         12   
3      4         c          c                620.0                          9   
4      5         a          a              29910.0                          4   

   CompetitionOpenSinceYear  Promo2  Promo2SinceWeek  Promo2SinceYear  \
0                      2008       0             <NA>             <NA>   
1                      2007       1               13             2010   
2                      2006       1               14             2011   
3                      2009       0             <NA>             <NA>   
4                      2015       0             <NA>             <NA>   

     PromoInterval  
0             <NA>  
1  Jan,Apr,Jul,Oct  
2  Jan,Apr,

In [14]:
print(store.dtypes)

Store                                 int64
StoreType                          category
Assortment                         category
CompetitionDistance                 float64
CompetitionOpenSinceMonth             Int64
CompetitionOpenSinceYear              Int64
Promo2                                int64
Promo2SinceWeek                       Int64
Promo2SinceYear                       Int64
PromoInterval                string[python]
dtype: object
